###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2018  by D. Koehn, heterogeneous models are from [this Jupyter notebook](https://nbviewer.jupyter.org/github/krischer/seismo_live/blob/master/notebooks/Computational%20Seismology/The Finite-Difference Method/fd_ac2d_heterogeneous.ipynb) by Heiner Igel ([@heinerigel](https://github.com/heinerigel)), Florian Wölfl and Lion Krischer ([@krischer](https://github.com/krischer)) which is a supplemenatry material to the book [Computational Seismology: A Practical Introduction](http://www.computational-seismology.org/), notebook style sheet by L.A. Barba, N.C. Clementi

In [1]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())

# Exercise: 2D acoustic FD modelling of the Marmousi-2 model

In this exercise, you have to apply all the knowledge about FD modelling, we covered so far. While the modelling examples in the last lesson where quite simple, we now calculate the 2D acoustic wave propagation in a more realistic problem called the **Marmousi-2 model**.
Developed in the 1990s by the French Petroleum Institute (IFP) ([Versteeg, 1994](https://library.seg.org/doi/abs/10.1190/1.1437051)), the Marmousi model is a widely used benchmark problem for seismic imaging and inversion techniques. Beside the original acoustic version of the model, an elastic version was developed by [Martin et al. (2006)](https://library.seg.org/doi/abs/10.1190/1.2172306).

The Marmousi-2 model consists of a 460 m thick water layer above an elastic subseafloor model.  The sediment model is very simple near the left and right boundaries but rather complex in the centre. At both sides, the subseafloor is approximately horizontally layered, while steep thrust faults are disturbing the layers in the centre of the model. Embedded in the thrust fault system and layers are small scale hydrocarbon reservoirs.

##### Exercise

Setup and model the 2D acoustic wave propagation in the Marmousi-2 model:

- Define the model discretization based on the given Marmousi-2 P-wave velocity model. Maximum wave propagation time should be 6 s 
- Calculate the central frequency $f_0$ of the source wavelet based on the grid dispersion criterion 

\begin{equation}
dx \le \frac{vp_{min}}{N_\lambda f_0}, \nonumber
\end{equation}

which you can use for the FD modelling run, based on the pre-defined $dx$ of the Marmousi-2 model , minimum P-wave velocity $vp_{min}$ and $N_\lambda = 4$ gridpoints per dominant wavelength.
- Start a modelling run for the Marmousi-2 model with the 3-point spatial FD operator. Place an airgun for a central shot at x = 5000 m at a depth = 40 m below the sea surface. Do not forget to calculate an appropriate time step $dt$
- Add an additional function `update_d2px_d2pz_5pt` to approximate the 2nd spatial derivatives by the 5-point FD operator in the 2D acoustic FD code derived [here](http://nbviewer.jupyter.org/github/daniel-koehn/Theory-of-seismic-waves-II/blob/master/04_FD_stability_dispersion/3_fd_taylor_operators.ipynb). Add an option `op` to switch between the 3-point and 5-point operator
- Start an additional modelling run for the Marmousi-2 model with the 5-point operator.
- Imagine you place an Ocean-Bottom-Cable (OBC) on the seafloor of the Marmousi-2 model. Calculate an OBC shot gather, by placing receivers at each gridpoint of the Cartesian model in x-direction in a depth of 460 m. Modify the FD code to record seismograms at each receiver position. Do not forget to return the seismograms from the modelling function.
- Plot and compare the seismograms produced by the 3- and 5-point operators, respectively.

In [ ]:
# Import Libraries 
# ----------------
import numpy as np
from numba import jit
import matplotlib
import matplotlib.pyplot as plt
from pylab import rcParams

# Ignore Warning Messages
# -----------------------
import warnings
warnings.filterwarnings("ignore")

from mpl_toolkits.axes_grid1 import make_axes_locatable

The elastic version of the Marmousi-2 model, together with FD modelled high frequency shot gather data is available from [here](http://www.agl.uh.edu/downloads/downloads.htm). The central part of the P-wave velocity model of the Marmousi-2 with the spatial discretization: 

$nx = 500$ gridpoints

$nz = 174$ gridpoints

$dx = dz = 20\; m$

is available as IEEE little endian binary file `marmousi_II_marine.vp` in the `marmousi-2` directory. It can be imported to Python with the following code snippet:

In [ ]:
# Import Marmousi-2 Vp model
# --------------------------

# DEFINE MODEL DISCRETIZATION HERE!
nx =        # number of grid points in x-direction
nz =        # number of grid points in z-direction
dx =        # spatial grid point distance in x-direction (m)
dz = dx        # spatial grid point distance in z-direction (m)

# Define model filename
name_vp = "marmousi-2/marmousi_II_marine.vp"

# Open file and write binary data to vp
f = open(name_vp)
data_type = np.dtype ('float32').newbyteorder ('<')
vp = np.fromfile (f, dtype=data_type)

# Reshape (1 x nx*nz) vector to (nx x nz) matrix 
vp = vp.reshape(nx,nz)

After reading the model into Python, we can take a look at it ...

In [ ]:
# Plot Marmousi-2 vp-model
# ------------------------

# Define xmax, zmax and model extension
xmax = nx * dx
zmax = nz * dz
extent = [0, xmax, zmax, 0]

fig = plt.figure(figsize=(12,3))  # define figure size

image = plt.imshow((vp.T)/1000, cmap=plt.cm.viridis, interpolation='nearest', 
                   extent=extent)

cbar = plt.colorbar(aspect=10, pad=0.02)
cbar.set_label('Vp [km/s]', labelpad=10)
plt.title('Marmousi-2 model')
plt.xlabel('x [m]')
plt.ylabel('z [m]')

In [ ]:
# Definition of modelling parameters
# ----------------------------------
# DEFINE MAXIMUM RECORDING TIME HERE!
tmax =   # maximum wave propagation time (s)

# DEFINE YOUR SHOT POSITION HERE!
xsrc =   # x-source position (m)
zsrc =   # z-source position (m)

# CALCULATE DOMINANT FREQUENCY OF THE SOURCE WAVELET HERE!
f0   =  # dominant frequency of the source (Hz)
print("f0 = ", f0, " Hz")
t0   = 4.0/f0   # source time shift (s)

isnap = 2  # snapshot interval (timesteps)

In [ ]:
@jit(nopython=True) # use JIT for C-performance
def update_d2px_d2pz_3pt(p, dx, dz, nx, nz, d2px, d2pz):
    
    for i in range(1, nx - 1):
        for j in range(1, nz - 1):
                
            d2px[i,j] = (p[i + 1,j] - 2 * p[i,j] + p[i - 1,j]) / dx**2                
            d2pz[i,j] = (p[i,j + 1] - 2 * p[i,j] + p[i,j - 1]) / dz**2
        
    return d2px, d2pz 

In [ ]:
# ADD THE SPATIAL 5-POINT OPERATORS HERE!
@jit(nopython=True) # use JIT for C-performance
def update_d2px_d2pz_5pt(p, dx, dz, nx, nz, d2px, d2pz):
    
        
    return d2px, d2pz 

In [ ]:
# Define simple absorbing boundary frame based on wavefield damping 
# according to Cerjan et al., 1985, Geophysics, 50, 705-708
def absorb(nx,nz):

    FW = 60     # thickness of absorbing frame (gridpoints)    
    a = 0.0053
    
    coeff = np.zeros(FW)
    
    # define coefficients in absorbing frame
    for i in range(FW):    
        coeff[i] = np.exp(-(a**2 * (FW-i)**2))

    # initialize array of absorbing coefficients
    absorb_coeff = np.ones((nx,nz))

    # compute coefficients for left grid boundaries (x-direction)
    zb=0 
    for i in range(FW):
        ze = nz - i - 1
        for j in range(zb,ze):
            absorb_coeff[i,j] = coeff[i]

    # compute coefficients for right grid boundaries (x-direction)        
    zb=0
    for i in range(FW):
        ii = nx - i - 1
        ze = nz - i - 1
        for j in range(zb,ze):
            absorb_coeff[ii,j] = coeff[i]

    # compute coefficients for bottom grid boundaries (z-direction)        
    xb=0 
    for j in range(FW):
        jj = nz - j - 1
        xb = j
        xe = nx - j
        for i in range(xb,xe):
            absorb_coeff[i,jj] = coeff[j]

    return absorb_coeff

In [ ]:
# FD_2D_acoustic code with JIT optimization
# -----------------------------------------
def FD_2D_acoustic_JIT(vp, dt,dx,dz,f0,xsrc,zsrc,op):        
    
    # calculate number of time steps nt 
    # ---------------------------------
    nt = (int)(tmax/dt)
    
    # locate source on Cartesian FD grid
    # ----------------------------------
    isrc = (int)(xsrc/dx)  # source location in grid in x-direction
    jsrc = (int)(zsrc/dz)  # source location in grid in x-direction    
    
    # Source time function (Gaussian)
    # -------------------------------
    src  = np.zeros(nt + 1)
    time = np.linspace(0 * dt, nt * dt, nt)

    # 1st derivative of Gaussian
    src  = -2. * (time - t0) * (f0 ** 2) * (np.exp(- (f0 ** 2) * (time - t0) ** 2)) 
    
    # define clip value: 0.1 * absolute maximum value of source wavelet
    clip = 0.5 * max([np.abs(src.min()), np.abs(src.max())]) / (dx*dz) * dt**2
    
    # Define absorbing boundary frame
    # -------------------------------    
    absorb_coeff = absorb(nx,nz)
    
    # Define squared vp-model
    # -----------------------        
    vp2 = vp**2
    
    # Initialize empty pressure arrays
    # --------------------------------
    p    = np.zeros((nx,nz)) # p at time n (now)
    pold = np.zeros((nx,nz)) # p at time n-1 (past)
    pnew = np.zeros((nx,nz)) # p at time n+1 (present)
    d2px = np.zeros((nx,nz)) # 2nd spatial x-derivative of p
    d2pz = np.zeros((nx,nz)) # 2nd spatial z-derivative of p 
    
    # INITIALIZE SEISMOGRAMS HERE! 
    # ----------------------------
        
    # Initalize animation of pressure wavefield 
    # -----------------------------------------    
    fig = plt.figure(figsize=(7,3))  # define figure size
    extent = [0.0,xmax,zmax,0.0]     # define model extension
    
    # Plot Vp-model
    image = plt.imshow((vp.T)/1000, cmap=plt.cm.gray, interpolation='nearest', 
                        extent=extent)    
    
    # Plot pressure wavefield movie
    image1 = plt.imshow(p.T, animated=True, cmap="RdBu", alpha=.75, extent=extent, 
                          interpolation='nearest', vmin=-clip, vmax=clip)    
    plt.title('Pressure wavefield')
    plt.xlabel('x [m]')
    plt.ylabel('z [m]')
           
    plt.ion()    
    plt.show(block=False)
    
    # Calculate Partial Derivatives
    # -----------------------------
    for it in range(nt):
    
        # FD approximation of spatial derivative by 3 point operator
        if(op==3):
            d2px, d2pz = update_d2px_d2pz_3pt(p, dx, dz, nx, nz, d2px, d2pz)
        
        # ADD FD APPROXIMATION OF SPATIAL DERIVATIVES BY 5 POINT OPERATOR HERE!
        #if(op==5):

        # Time Extrapolation
        # ------------------
        pnew = 2 * p - pold + vp2 * dt**2 * (d2px + d2pz)

        # Add Source Term at isrc
        # -----------------------
        # Absolute pressure w.r.t analytical solution
        pnew[isrc,jsrc] = pnew[isrc,jsrc] + src[it] / (dx * dz) * dt ** 2
        
        # Apply absorbing boundary frame
        # ------------------------------
        p *= absorb_coeff
        pnew *= absorb_coeff
        
        # Remap Time Levels
        # -----------------
        pold, p = p, pnew
        
        # WRITE SEISMOGRAMS HERE!
    
        # display pressure snapshots 
        if (it % isnap) == 0:            
            image1.set_data(p.T)
            fig.canvas.draw()
    
    # DO NOT FORGET TO RETURN THE SEISMOGRAM HERE!      

## Model wave propagation in the Marmousi-2 model with 2D acoustic FD code

Time to model acoustic wave propagation in the Marmouisi-2 model using the 3-point operator. We only have to define the timestep $dt$:

In [ ]:
# Run 2D acoustic FD modelling with 3-point spatial operater
# ----------------------------------------------------------
%matplotlib notebook
op = 3  # define spatial FD operator (3-point) 

# DEFINE TIME STEP HERE!
dt =  # time step (s)

FD_2D_acoustic_JIT(vp,dt,dx,dz,f0,xsrc,zsrc,op)

In [ ]:
# Run 2D acoustic FD modelling with 5-point spatial operater
# ----------------------------------------------------------
%matplotlib notebook
op = 5  # define spatial FD operator (5-point) 

# DEFINE TIME STEP HERE!
dt   =  # time step (s)

FD_2D_acoustic_JIT(vp,dt,dx,dz,f0,xsrc,zsrc,op)

In [ ]:
%matplotlib notebook
# PLOT YOUR MODELLED OBC SHOT GATHER HERE!
clip_seis = 1e-7
extent_seis = [0.0,xmax/1000,tmax,0.0]

plt.subplot(121)
plt.imshow(seis_marm_3pt.T, cmap=plt.cm.gray, aspect=2, vmin=-clip_seis, 
                   vmax=clip_seis, extent=extent_seis)

plt.title('3-point operator')
plt.xlabel('x [km]')
plt.ylabel('t [s]')


ax = plt.subplot(122)
plt.imshow(seis_marm_5pt.T, cmap=plt.cm.gray, aspect=2, vmin=-clip_seis, 
                   vmax=clip_seis, extent=extent_seis)
ax.set_yticks([]) 
plt.title('5-point operator')
plt.xlabel('x [km]')
#plt.ylabel('t [s]')
plt.tight_layout()
plt.show()

## What we learned:

- How to model wave propgation in the complex Marmousi-2 model. Now you can model everything ...